In [53]:
# Import Libraries
import pandas as pd
import math
import numpy as np
import sklearn
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras import layers

from sklearn import preprocessing

from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split

from sklearn.svm import SVR
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.linear_model import LinearRegression

In [54]:
# Data loading
data = pd.read_csv('/content/ISCTR.IS (1).csv', parse_dates=True, index_col=0)

data = data.dropna()

data['High_Low_per'] = (data['High'] - data['Close']) / data['Close'] * 100
data['Per_change'] = (data['Open'] - data['Open']) / data['Close'] * 100
data = data[['Adj Close', 'High_Low_per', 'Per_change', 'Volume']]
label_col = 'Adj Close'

ceil_data = int(math.ceil(0.002*len(data)))
data['label'] = data[label_col].shift(-ceil_data)

X = data.drop(['label'], axis=1).values
X = preprocessing.scale(X)
X = X[:-ceil_data]

# Predict on next days
forecast = X[-ceil_data-7:-ceil_data]

y = data['label'].values

# Drop rows with missing values
data.dropna(inplace=True)

# Convert label column to numpy array
y = data['label'].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

<ipython-input-54-36eae4ab8f37>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['label'] = data[label_col].shift(-ceil_data)
<ipython-input-54-36eae4ab8f37>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.dropna(inplace=True)


In [55]:
svr = SVR(kernel='rbf', C=1000, gamma=0.1)
svr.fit(X_train, y_train)

# Predict the test data
prediction_svm = svr.predict(X_test)

# Predict the next 7 days of data
svm_next7days = svr.predict(forecast)

print('Predictions on Test Data:', prediction_svm)
print('Predictions for Next 7 Days:', svm_next7days)

Predictions on Test Data: [ 2.73927915  2.44152966  2.14911599  1.8714293   1.88303448  2.73860714
  2.53630591  1.71343963  2.65185341  2.21238369  2.68715635  1.97493051
  1.97317629  1.81372     2.28874499  2.29479398  3.04828522  1.78547751
  1.69857633  2.48433859  2.50425593  2.41848487  1.74236257  2.25345195
  5.02433283  2.53148855  2.58112361  2.11572927  1.85790575  2.69392985
  1.73934363  2.19164909  3.74205485  7.49722286  1.88701753  4.51974366
  1.76831453  2.24283344  2.02240422  2.7186701   2.12709075  2.04928306
  3.05763198  3.08084658  2.16443536  3.12041842  7.41460813  2.62173654
  2.1264686   2.47510083  1.76898278  8.51832762  2.5159862   3.36827931
  3.06170606  2.09199255  3.49226894  2.15414181  1.93641815  6.09843391
  2.49316966  8.87966225  1.71089886  2.54541162  2.25813242  8.99154833
  3.92924427  3.58803497  2.0804425   2.19642167  1.98196003  2.83960838
  3.39120985  2.90405299  1.87879921  2.2743204   1.90730902  3.15182668
  2.36032016  3.65178268 

In [56]:
svm_accuracy = svr.score(X_test, y_test)

print(svm_accuracy*100)

91.89448235523395


In [57]:
lr = LinearRegression()
lr.fit(X_train, y_train)

lr_prediction = lr.predict(X_test)

# Predict the next 7 days of data
lr_next7days = lr.predict(forecast)

print('Predictions on Test Data:', lr_prediction)
print('Predictions for Next 7 Days:', lr_next7days)

Predictions on Test Data: [ 2.7533381   2.43614415  2.1813279   1.84536307  1.89728911  2.7466949
  2.52540655  1.6727137   2.64677114  2.24065263  2.7146231   1.93322041
  1.91992063  1.75722835  2.31934008  2.32584433  3.06451089  1.78838791
  1.80973464  2.50021375  2.5480729   2.42757764  1.80584346  2.26299543
  4.95647135  2.52341292  2.60419471  2.12980566  1.85707978  2.7246819
  1.7869844   2.23565988  3.7610811  12.9543003   1.88877445  4.39062999
  1.73513591  2.26350464  1.98108592  2.72039341  2.15934481  2.08601706
  3.06456721  3.09314417  2.19370349  3.12838552  7.78930961  2.65936075
  2.13545144  2.51201918  1.78510452 12.68324198  2.53107805  3.1902704
  3.06420247  2.08999558  3.49467297  2.1035306   1.91705333  6.08340136
  2.5255168   8.12708143  1.80829338  2.58571451  2.25576712  8.39918257
  3.96052982  3.567904    2.05771662  2.18041046  1.99782291  2.86243834
  3.51127779  2.63727591  1.88302201  2.26165845  1.91794928  3.24364584
  2.37549549  3.69230068  2.

In [58]:
lr_accuracy = lr.score(X_test, y_test)

print(lr_accuracy*100)

95.57488118845772


In [59]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs = 40, batch_size = 30, validation_data = (X_test, y_test))

Epoch 1/40
55/55 [==============================] - 6s 29ms/step - loss: 6.2317 - val_loss: 1.1125
Epoch 2/40
55/55 [==============================] - 1s 11ms/step - loss: 1.0638 - val_loss: 0.7835
Epoch 3/40
55/55 [==============================] - 1s 11ms/step - loss: 0.7960 - val_loss: 0.6501
Epoch 4/40
55/55 [==============================] - 1s 12ms/step - loss: 0.6772 - val_loss: 0.5634
Epoch 5/40
55/55 [==============================] - 1s 15ms/step - loss: 0.5561 - val_loss: 0.4219
Epoch 6/40
55/55 [==============================] - 1s 14ms/step - loss: 0.5074 - val_loss: 0.3255
Epoch 7/40
55/55 [==============================] - 1s 12ms/step - loss: 0.3811 - val_loss: 0.2794
Epoch 8/40
55/55 [==============================] - 1s 11ms/step - loss: 0.3504 - val_loss: 0.3447
Epoch 9/40
55/55 [==============================] - 1s 11ms/step - loss: 0.3141 - val_loss: 0.2260
Epoch 10/40
55/55 [==============================] - 1s 11ms/step - loss: 0.2883 - val_loss: 0.2033
Epoch 11/

In [60]:
dnn_prediction = model.predict(X_test)

# Predict for the next 7 days data
dnn_next7days = model.predict(forecast)

print('Predictions on Test Data:', dnn_prediction)
print('Predictions for Next 7 Days:', dnn_next7days)

1/1 [==============================] - 0s 32ms/step
Predictions on Test Data: [[ 2.8113139]
 [ 2.5084693]
 [ 2.1349797]
 [ 1.9498287]
 [ 1.8976365]
 [ 2.836773 ]
 [ 2.6933515]
 [ 1.673283 ]
 [ 2.8028307]
 [ 2.2039118]
 [ 2.6187785]
 [ 2.062015 ]
 [ 2.1537333]
 [ 1.918947 ]
 [ 2.2692845]
 [ 2.2195964]
 [ 3.228152 ]
 [ 1.8471026]
 [ 1.7747118]
 [ 2.4806948]
 [ 2.4738362]
 [ 2.4199731]
 [ 1.8341602]
 [ 2.2912068]
 [ 5.506311 ]
 [ 2.7222598]
 [ 2.5355852]
 [ 2.1628354]
 [ 1.8789688]
 [ 2.6733623]
 [ 1.7494427]
 [ 2.2027705]
 [ 3.8688667]
 [11.096847 ]
 [ 1.9390448]
 [ 4.741055 ]
 [ 1.8215642]
 [ 2.2738266]
 [ 2.1152291]
 [ 2.8611252]
 [ 2.0863261]
 [ 2.0496588]
 [ 3.1738148]
 [ 3.2517965]
 [ 2.123376 ]
 [ 3.294668 ]
 [ 9.288365 ]
 [ 2.6519969]
 [ 2.175155 ]
 [ 2.5442698]
 [ 1.7278136]
 [11.768876 ]
 [ 2.5315812]
 [ 3.1634796]
 [ 3.2189064]
 [ 2.160794 ]
 [ 3.482021 ]
 [ 2.252034 ]
 [ 2.0067115]
 [ 7.068617 ]
 [ 2.5203414]
 [ 9.160234 ]
 [ 1.8180813]
 [ 2.6246693]
 [ 2.312349 ]
 [ 9.705707 